In [2]:
from fastai import text
import pandas as pd
import regexTokenizer as rgt
from glob import glob
import torch.sparse as ts
import torch
import fastai.text.all as faitext
import fastai as fa

print(torch.cuda.is_available(), torch.cuda.get_device_name(0))

True NVIDIA GeForce GTX 1650 Ti


In [4]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

In [2]:
faitext.defaults.text_proc_rules = []

In [3]:
from regexTokenizer import regexKmerTokenizer


In [4]:
tok = regexKmerTokenizer(1000, overlapping= False, protein = True)
%time df = tok.seq2pandas('human genome/GRCh38_latest_protein.faa.gz', docformat = 'fasta',from_file=True)
def sppliter(name):
    left, right = name.find(' '), name.rfind('[')
    return [name[:left], name[left:right], name[right:]]
df[['ID', 'name', 'species']] = [sppliter(i) for i in df.name]
df

Wall time: 1.84 s


,name,sequence,ID,species
0,alpha-2-macroglobulin isoform a precursor,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEKGCVLLSYLNETVTVSASLESVRGNRSLFTDLEAENDVLHCVAFAVPKSSSNEEVMFLTVQVKGPTQEFKKRTTVMVKNEDSLVFVQTDKSIYKPGQTVKFRVVSMDENFHPLNELIPLVYIQDPKGNRIAQWQSFQLEGGLKQFSFPLSSEPFQGSYKVVVQKKSGGRTEHPFTVEEFVLPKFEVQVTVPKIITILEEEMNVSVCGLYTYGKPVPGHVTVSICRKYSDASDCHGEDSQAFCEKFSGQLNSHGCFYQQVKTKVFQLKRKEYEMKLHTEAQIQEEGTVVELTGRQSSEITRTITKLSFVKVDSHFRQGIPFFGQVRLVDGKGVPIPNKVIFIRGNEANYYSNATTDEHGLVQFSINTTNVMGTSLTVRVNYKDRSPCYGYQWVSEEHEEAHHTAYLVFSPSKSFVHLEPMSHELPCGHTQTVQAHYILNGGTLLGLKKLSFYYLIMAKGGIVRTGTHGLLVKQEDMKGHFSISIPVKSDIAPVARLLIYAVLPTGDVIGDSAKYDVENCLANKVDLSFSPSQSLPASHAHLRVTAAPQSVCA...,NP_000005.3,[Homo sapiens]
1,arylamine N-acetyltransferase 2,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQAMELGLEAIFDHIVRRNRGGWCLQVNQLLYWALTTIGFQTTMLGGYFYIPPVNKYSTGMVHLLLQVTIDGRNYIVDAGSGSSSQMWQPLELISGKDQPQVPCIFCLTEERGIWYLDQIRREQYITNKEFLNSHLLPKKKHQKIYLFTLEPRTIEDFESMNTYLQTSPTSSFITTSFCSLQTPEGVYCLVGFILTYRKFNYKDNTDLVEFKTLTEEEVEEVLRNIFKISLGRNLVPKPGDGSLTI,NP_000006.2,[Homo sapiens]
2,"medium-chain specific acyl-CoA dehydrogenase, mitochondrial isoform a precursor",MAAGFGRCCRVLRSISRFHWRSQHTKANRQREPGLGFSFEFTEQQKEFQATARKFAREEIIPVAAEYDKTGEYPVPLIRRAWELGLMNTHIPENCGGLGLGTFDACLISEELAYGCTGVQTAIEGNSLGQMPIIIAGNDQQKKKYLGRMTEEPLMCAYCVTEPGAGSDVAGIKTKAEKKGDEYIINGQKMWITNGGKANWYFLLARSDPDPKAPANKAFTGFIVEADTPGIQIGRKELNMGQRCSDTRGIVFEDVKVPKENVLIGDGAGFKVAMGAFDKTRPVVAAGAVGLAQRALDEATKYALERKTFGKLLVEHQAISFMLAEMAMKVELARMSYQRAAWEVDSGRRNTYYASIAKAFAGDIANQLATDAVQILGGNGFNTEYPVEKLMRDAKIYQIYEGTSQIQRLIVAREHIDKYKN,NP_000007.1,[Homo sapiens]
3,"short-chain specific acyl-CoA dehydrogenase, mitochondrial isoform 1 precursor",MAAALLARASGPARRALCPRAWRQLHTIYQSVELPETHQMLLQTCRDFAEKELFPIAAQVDKEHLFPAAQVKKMGGLGLLAMDVPEELGGAGLDYLAYAIAMEEISRGCASTGVIMSVNNSLYLGPILKFGSKEQKQAWVTPFTSGDKIGCFALSEPGNGSDAGAASTTARAEGDSWVLNGTKAWITNAWEASAAVVFASTDRALQNKGISAFLVPMPTPGLTLGKKEDKLGIRGSSTANLIFEDCRIPKDSILGEPGMGFKIAMQTLDMGRIGIASQALGIAQTALDCAVNYAENRMAFGAPLTKLQVIQFKLADMALALESARLLTWRAAMLKDNKKPFIKEAAMAKLAASEAATAISHQAIQILGGMGYVTEMPAERHYRDARITEIYEGTSEIQRLVIAGHLLRSYRS,NP_000008.1,[Homo sapiens]
4,"very long-chain specific acyl-CoA dehydrogenase, mitochondrial isoform 1 precursor",MQAARMAASLGRQLLRLGGGSSRLTALLGQPRPGPARRPYAGGAAQLALDKSDSHPSDALTRKKPAKAESKSFAVGMFKGQLTTDQVFPYPSVLNEEQTQFLKELVEPVSRFFEEVNDPAKNDALEMVEETTWQGLKELGAFGLQVPSELGGVGLCNTQYARLVEIVGMHDLGVGITLGAHQSIGFKGILLFGTKAQKEKYLPKLASGETVAAFCLTEPSSGSDAASIRTSAVPSPCGKYYTLNGSKLWISNGGLADIFTVFAKTPVTDPATGAVKEKITAFVVERGFGGITHGPPEKKMGIKASNTAEVFFDGVRVPSENVLGEVGSGFKVAMHILNNGRFGMAAALAGTMRGIIAKAVDHATNRTQFGEKIHNFGLIQEKLARMVMLQYVTESMAYMVSANMDQGATDFQIEAAISKIFGSEAAWKVTDECIQIMGGMGFMKEPGVERVLRDLRIFRIFEGTNDILRLFVALQGCMDKGKELSGLGSALKNPFGNAGLLLGEAGKQLRRRAGLGSGLSLSGLVHPELSRSGELAVRALEQFATVVEAKLIKHKKGIVNEQFLLQRLADGAIDLYAMVVVLSRASRSLSEGHPTA...,NP_000009.1,[Homo sapiens]
...,...,...,...,...
115346,NADH dehydrogenase subunit 4L (mitochondrion),MPLIYMNIMLAFTISLLGMLVYRSHLMSSLLCLEGMMLSLFIMATLMTLNTHSLLANIVPIAMLVFAACEAAVGLALLVSISNTYGLDYVHNLNLLQC,YP_003024034.1,[Homo sapiens]
115347,NADH dehydrogenase subunit 4 (mitochondrion),MLKLIVPTIMLLPLTWLSKKHMIWINTTTHSLIISIIPLLFFNQINNNLFSCSPTFSSDPLTTPLLMLTTWLLPLTIMASQRHLSSEPLSRKKLYLSMLISLQISLIMTFTATELIMFYIFFETTLIPTLAIITRWGNQPERLNAGTYFLFYTLVGSLPLLIALIYTHNTLGSLNILLLTLTAQELSNSWANNLMWLAYTMAFMVKMPLYGLHLWLPKAHVEAPIAGSMVLAAVLLKLGGYGMMRLTLILNPLTKHMAYPFLVLSLWGMIMTSSICLRQTDLKSLIAYSSISHMALVVTAILIQTPWSFTGAVILMIAHGLTSSLLFCLANSNYERTHSRIMILSQGLQTLLPLMAFWWLLASLANLALPPTINLLGELSVLVTTFSWSNITLLLTGLNMLVTALYSLYMFTTTQWGSLTHHINNMKPSFTRENTLMFMHLSPILLLSLNPDIITGFSS,YP_003024035.1,[Homo sapiens]
115348,NADH dehydrogenase subunit 5 (mitochondrion),MTMHTTMTTLTLTSLIPPILTTLVNPNKKNSYPHYVKSIVASTFIISLFPTTMFMCLDQEVIISNWHWATTQTTQLSLSFKLDYFSMMFIPVALFVTWSIMEFSLWYMNSDPNINQFFKYLLIFLITMLILVTANNLFQLFIGWEGVGIMSFLLISWWYARADANTAAIQAILYNRIGDIGFILALAWFILHSNSWDPQQMALLNANPSLTPLLGLLLAAAGKSAQLGLHPWLPSAMEGPTPVSALLHSSTMVVAGIFLLIRFHPLAENSPLIQTLTLCLGAITTLFAAVCALTQNDIKKIVAFSTSSQLGLMMVTIGINQPHLAFLHICTHAFFKAMLFMCSGSIIHNLNNEQDIRKMGGLLKTMPLTSTSLTIGSLALAGMPF

In [ ]:
tokAA = faitext.SentencePieceTokenizer(sp_model = 'humanProtAA.model')
print(list(tokAA(['MCGF', 'MCHFO'])))
tokName = faitext.SentencePieceTokenizer(sp_model = 'humanProtNames.model')
print(list(tokName(['NADH dehydrogenase subunit 4L', 'medium-chain specific acyl-CoA dehydrogenase, mitochondrial isoform a precursor'])))

In [ ]:
class LongVariableKmerTokenizer:
    def __init__(self, modelname = 'sarsKmer.model', minsize = 4):
        import sentencepiece as spm
        self.tok = spm.SentencePieceProcessor(model_file=modelname)
        self.minsize = minsize
    def __call__(self, items):
        for t in items: 
            yield [i.strip('▁') for i in self.tok.EncodeAsPieces(t)] #len(i.strip('▁'))>= self.minsize
                    
LVKTok = LongVariableKmerTokenizer(modelname = 'humanProtAA.model', minsize = 1)

In [ ]:
%%time
dls_lm_name = faitext.TextDataLoaders.from_df(df.sample(50000), tok_tfm=tokName,is_lm=True, valid_pct=0.1, text_col= 'name', rules =[], seq_len = 20)
dls_lm_name.show_batch(max_n=20)


In [ ]:
%%time
dls_lm = faitext.TextDataLoaders.from_df(df.sample(50000), tok_tfm=tokAA,is_lm=True, valid_pct=0.1, text_col= 'sequence', rules =[], seq_len = 30 )
dls_lm.show_batch(max_n=20)


In [ ]:
dls_lm.cuda()
#dls_lm_name.cuda()

In [ ]:
learn = faitext.language_model_learner(dls_lm, faitext.AWD_LSTM, metrics=[faitext.accuracy, faitext.Perplexity()], wd=0.1).to_fp16()
#learn_name = faitext.language_model_learner(dls_lm_name, faitext.AWD_LSTM, metrics=[faitext.accuracy, faitext.Perplexity()], wd=0.1).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn_name.unfreeze()
learn_name.fit_one_cycle(5, .0063095735386013985)

In [ ]:
#learn.save('Sarscov2fastai')
#learn = learn.load('Sarscov2fastai')

In [ ]:
learn = learn.load('Sarscov2fastai')
TEXT = "GGTAACAAACCAA"
N_WORDS = 40
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=2).replace(' ', '')   for _ in range(N_SENTENCES)]
for num,i in enumerate(preds): print('>{}\n'.format(num)+ i)

In [ ]:
TEXT = "zinc finger"
N_WORDS = 20
N_SENTENCES = 5
preds = [learn_name.predict(TEXT, N_WORDS, temperature=.5).replace(' ', '')   for _ in range(N_SENTENCES)]
for num,i in enumerate(preds): print('>{}\n'.format(num)+ i)

In [ ]:

        
list(LVKTok(['CGATCGATCGTACGTAGTCAGTGCATCGATGCATGCATGACTCAGA', 'ACGTACGTACGTACGCAAAAAAAAAATGGCATGCATCA']))